In [35]:
import xarray as xr
import numpy as np

## Prepare data

In [36]:
infile = '/g/data/xv83/dbi599/precip_c5-d60-pX-f6_19861101-19871101_atmos_isobaric_daily.zarr.zip'

In [37]:
ds = xr.open_zarr(infile)

In [38]:
ds

<xarray.Dataset>
Dimensions:    (ensemble: 96, init_date: 2, lat: 19, lead_time: 3653, lon: 19)
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) datetime64[ns] 1986-11-01 1987-11-01
  * lat        (lat) float64 -45.51 -43.48 -41.46 ... -13.15 -11.12 -9.101
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 ... 3647 3648 3649 3650 3651 3652
  * lon        (lon) float64 111.2 113.8 116.2 118.8 ... 148.8 151.2 153.8 156.2
    time       (lead_time, init_date) datetime64[ns] dask.array<chunksize=(3653, 2), meta=np.ndarray>
Data variables:
    precip     (init_date, lead_time, ensemble, lat, lon) float32 dask.array<chunksize=(1, 28, 96, 19, 19), meta=np.ndarray>

In [40]:
da = ds['precip'] * 86400
da.attrs['units'] = 'mm'
da

<xarray.DataArray 'precip' (init_date: 2, lead_time: 3653, ensemble: 96, lat: 19, lon: 19)>
dask.array<mul, shape=(2, 3653, 96, 19, 19), dtype=float64, chunksize=(1, 28, 96, 19, 19), chunktype=numpy.ndarray>
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) datetime64[ns] 1986-11-01 1987-11-01
  * lat        (lat) float64 -45.51 -43.48 -41.46 ... -13.15 -11.12 -9.101
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 ... 3647 3648 3649 3650 3651 3652
  * lon        (lon) float64 111.2 113.8 116.2 118.8 ... 148.8 151.2 153.8 156.2
    time       (lead_time, init_date) datetime64[ns] dask.array<chunksize=(3653, 2), meta=np.ndarray>
Attributes:
    units:    mm

In [41]:
def reindex_forecast(ds, dropna=False):
    """Switch out lead_time axis for time axis (or vice versa) in a forecast dataset."""
    
    if 'lead_time' in ds.dims:
        index_dim = 'lead_time'
        reindex_dim = 'time'
    elif 'time' in ds.dims:
        index_dim = 'time'
        reindex_dim = 'lead_time'
    else:
        raise ValueError("Neither a time nor lead_time dimension can be found")
    swap = {index_dim: reindex_dim}
    to_concat = []
    for init_date in ds['init_date']:
        fcst = ds.sel({'init_date': init_date})
        fcst = fcst.where(fcst[reindex_dim].notnull(), drop=True)
        fcst = fcst.assign_coords({'lead_time': fcst['lead_time'].astype(int)})
        to_concat.append(fcst.swap_dims(swap))
    concat = xr.concat(to_concat, dim='init_date')
    if dropna:
        return concat.where(concat.notnull(), drop=True)
    else:
        return concat

In [42]:
da = reindex_forecast(da)

In [43]:
da

,Array,Chunk
Bytes,2.23 GB,7.76 MB
Shape,"(2, 4018, 96, 19, 19)","(1, 28, 96, 19, 19)"
Count,5487 Tasks,522 Chunks
Type,float64,numpy.ndarray


In [16]:
def sum_min_samples(ds, dim, min_samples):
    """Return sum only if there are more than min_samples along dim."""
    
    s = ds.sum(dim, skipna=False)
    # Reference to final lead in sample
    if 'lead_time' in ds.coords:
        if dim in ds['lead_time'].dims:
            l = ds['lead_time'].max(dim, skipna=False)
            s = s.assign_coords({'lead_time': l if len(ds[dim]) >= min_samples else np.nan*l})
    if 'lead_year' in ds.coords:
        l = ds['lead_year'].max(dim, skipna=False)
        s = s.assign_coords({'lead_year': l if len(ds[dim]) >= min_samples else np.nan*l})
        
    return s if len(ds[dim]) >= min_samples else np.nan*s

In [44]:
# total annual (Dec- Nov) precip (mm)
da = da.resample(time="A-DEC", label='right').apply(sum_min_samples, dim='time', min_samples=365)

In [45]:
da

<xarray.DataArray 'precip' (init_date: 2, time: 12, ensemble: 96, lat: 19, lon: 19)>
dask.array<transpose, shape=(2, 12, 96, 19, 19), dtype=float64, chunksize=(1, 1, 96, 19, 19), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 1986-12-31 1987-12-31 ... 1997-12-31
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) datetime64[ns] 1986-11-01 1987-11-01
  * lat        (lat) float64 -45.51 -43.48 -41.46 ... -13.15 -11.12 -9.101
  * lon        (lon) float64 111.2 113.8 116.2 118.8 ... 148.8 151.2 153.8 156.2
    lead_time  (time, init_date) float64 nan nan 425.0 nan ... 3.348e+03 nan nan

In [46]:
# spatial average of the total annual rainfall

# local_funcs.py used to have a function for this, which applied a region mask.
# That function doesn't exist anymore and I'm not sure how to access the region mask file
#
#def average_region(ds, region_mask):
#    return ds.where(region_mask).mean(['lat','lon'])

da = da.mean(['lat', 'lon'])
da

<xarray.DataArray 'precip' (init_date: 2, time: 12, ensemble: 96)>
dask.array<mean_agg-aggregate, shape=(2, 12, 96), dtype=float64, chunksize=(1, 1, 96), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 1986-12-31 1987-12-31 ... 1997-12-31
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) datetime64[ns] 1986-11-01 1987-11-01
    lead_time  (time, init_date) float64 nan nan 425.0 nan ... 3.348e+03 nan nan

## Do the bias correction

In [ ]:
%%time
fcst = f5_precip_ann
obsv = awap_precip_ann

# Force leads to integers so fcst can be reindexed

# def reindex_forecast_hack(fcst, dropna=False):
#    """ Reindex forecast with daily lead_time by first converting lead_time to annual frequency 
#    """
#    hack_leads = np.floor(((fcst.lead_time-DAYS_TO_ZERO_LEAD) / 365))
#    return reindex_forecast(fcst.assign_coords({'lead_time': hack_leads}), dropna=dropna)

fcst_hack = lf.reindex_forecast_hack(fcst)

stack_dates = fcst_hack.sel(
    init_date=slice(
        (pd.to_datetime(period[0]) - pd.DateOffset(years=10)).strftime('%Y-%m-%d'),
         pd.to_datetime(period[1]).strftime('%Y-%m-%d'))).init_date.values

obsv_stacked = lf.stack_by_init_date(
    obsv, stack_dates, max(fcst_hack.lead_time.values)+2) 
# Shift leads back by one so that obsv dates match fcst dates
obsv_stacked = obsv_stacked.assign_coords({'lead_time': obsv_stacked.lead_time-1}) \
                           .where(obsv_stacked.lead_time > 0, drop=True)

mask = (fcst_hack.time >= period[0]) & (fcst_hack.time <= period[1])
    
fcst_clim = fcst_hack.mean('ensemble').where(mask, drop=True).groupby('init_date.month').mean('init_date')
obsv_clim = obsv_stacked.where(mask, drop=True).groupby('init_date.month').mean('init_date')

model_mulbias_precip_ann = (fcst_clim / obsv_clim).chunk('auto').rename('precip_ann')
model_mulbias_precip_ann = model_mulbias_precip_ann.xc.cache(f'CAFE-model-mulbias_precip-ann-accl_{REGION_NAME}.zarr', clobber=clobber)

model_addbias_precip_ann = (fcst_clim - obsv_clim).chunk('auto').rename('precip_ann')
model_addbias_precip_ann = model_addbias_precip_ann.xc.cache(f'CAFE-model-addbias_precip-ann-accl_{REGION_NAME}.zarr', clobber=clobber)